In [542]:
import requests
import time
import datetime as dt
import pandas as pd

import hashlib
import hmac
import base64

import json

from pprint import pprint 

from pandas.io.json import json_normalize

import numpy
import logging

In [543]:
# Current time stamp
timestamp = int(time.time())

In [544]:
logging.basicConfig(level=logging.INFO, 
                    format=' %(asctime)s - %(levelname)s - %(message)s')

In [545]:
# Date
today_date = dt.date.today() + dt.timedelta(days=0)
yesterday_date = dt.date.today() + dt.timedelta(days=-1)
seven_days_before_date = dt.date.today() + dt.timedelta(days=-7)

In [546]:
# Shop Parameter
partner_id = 12155
shopid = 23070969
shop_key = 'de5e924b8ed680bc9b22a6c402058154340333ff05641a38069f2856a2f3e24e'

In [547]:
# get_auth_signature
def get_auth_signature(url, payload):
    # API Key
    shopee_secret = shop_key
    # Post Message
    post_message = url + '|' + json.dumps(payload)
    # HMAC-SHA256
    message = bytes(post_message, 'utf-8')
    secret = bytes(shopee_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    auth_signature = hash.hexdigest()
    return auth_signature

In [548]:
# URL
url = 'https://partner.shopeemobile.com/api/v1/items/get'

In [549]:
# get all item
max_page = 200
pagination_entries_per_page = 100
i = 0

try:
    while i <= max_page:
        # start entry
        start_entry = i * pagination_entries_per_page

        # Parameter
        pagination_offset = start_entry

        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "pagination_offset":pagination_offset,
                   "pagination_entries_per_page":pagination_entries_per_page}
        # Auth Signature
        Auth_signature = get_auth_signature(url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_list = json_normalize(parsed['items'])
        # 排除update_time
        selected_column = ['item_id', 'shopid', 'status']
        item_list = item_list[selected_column]

        # 合并
        if i == 0:
            all_item_list = item_list
        else:
            all_item_list = all_item_list.append(item_list)

        i += 1
        logging.info('Retrieving item from page ' + str(i))
except:
    pass

# 去重
logging.info('Remove duplicate records.')
all_item_list = all_item_list.drop_duplicates('item_id').reset_index()
logging.info('Retrieved item list.')
print(all_item_list.head())

 2018-05-25 16:07:04,470 - INFO - Retrieving item from page 1
 2018-05-25 16:07:04,712 - INFO - Retrieving item from page 2
 2018-05-25 16:07:04,914 - INFO - Remove duplicate records.
 2018-05-25 16:07:04,916 - INFO - Retrieved item list.


   index     item_id    shopid  status
0      0  1121446661  23070969  NORMAL
1      1  1121455576  23070969  NORMAL
2      2  1121423066  23070969  NORMAL
3      3   725349781  23070969  NORMAL
4      4   725351767  23070969  NORMAL


In [550]:
# get item detail
item_detail_url = 'https://partner.shopeemobile.com/api/v1/item/get'

In [551]:
item_detail_list = pd.DataFrame()

for key, item in all_item_list.iterrows():
    try:
        item_id = item['item_id']
        logging.info('Now retrieving ' + str(item_id) + ' detail.')
        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "item_id": item_id}
        # Auth Signature
        Auth_signature = get_auth_signature(item_detail_url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(item_detail_url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_detail = json_normalize(parsed['item'])
        item_detail_list = item_detail_list.append(item_detail)
        
        # print(item_detail)
        logging.info('Retrieved ' + str(item_id) + ' detail.')
    except:
        pass
    
# convert timestamp to date
item_detail_list['create_time'] = pd.to_datetime(item_detail_list['create_time'], unit='s').dt.date
item_detail_list['update_time'] = pd.to_datetime(item_detail_list['update_time'], unit='s').dt.date
item_detail_list['days_since_live'] = (today_date - item_detail_list['create_time']).dt.days

 2018-05-25 16:07:04,942 - INFO - Now retrieving 1121446661 detail.
 2018-05-25 16:07:05,344 - INFO - Retrieved 1121446661 detail.
 2018-05-25 16:07:05,345 - INFO - Now retrieving 1121455576 detail.
 2018-05-25 16:07:05,764 - INFO - Retrieved 1121455576 detail.
 2018-05-25 16:07:05,765 - INFO - Now retrieving 1121423066 detail.
 2018-05-25 16:07:06,229 - INFO - Retrieved 1121423066 detail.
 2018-05-25 16:07:06,230 - INFO - Now retrieving 725349781 detail.
 2018-05-25 16:07:06,933 - INFO - Retrieved 725349781 detail.
 2018-05-25 16:07:06,934 - INFO - Now retrieving 725351767 detail.
 2018-05-25 16:07:07,306 - INFO - Retrieved 725351767 detail.
 2018-05-25 16:07:07,307 - INFO - Now retrieving 725352772 detail.
 2018-05-25 16:07:07,586 - INFO - Retrieved 725352772 detail.
 2018-05-25 16:07:07,586 - INFO - Now retrieving 288208978 detail.
 2018-05-25 16:07:08,246 - INFO - Retrieved 288208978 detail.
 2018-05-25 16:07:08,247 - INFO - Now retrieving 289817750 detail.
 2018-05-25 16:07:08,740

In [552]:
item_detail_list

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,sales,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live
0,"[{'attribute_name': '型號', 'is_mandatory': True...",5804,0,2018-05-02,TWD,3,Fashion Patented High Quality Anti Slip Crash ...,True,[http://f.shopee.tw/file/a7a43d76b0361451988df...,1121446661,...,0,23070969,NORMAL,5000,2018-05-04,"[{'status': 'MODEL_NORMAL', 'original_price': ...",2,0.10,[],23
0,"[{'attribute_name': '型號', 'is_mandatory': True...",5804,0,2018-05-02,TWD,3,Lovely Cartoon Anti Slip Crash Proof Phone Cas...,True,[http://f.shopee.tw/file/43dd056d8d0a1fb574ce1...,1121455576,...,0,23070969,NORMAL,8000,2018-05-02,"[{'status': 'MODEL_NORMAL', 'original_price': ...",4,0.10,[],23
0,"[{'attribute_name': '型號', 'is_mandatory': True...",5804,0,2018-05-02,TWD,3,Fashion Patented High Quality Anti Slip Crash ...,True,[http://f.shopee.tw/file/e54a6fd53d6872355c7d3...,1121423066,...,0,23070969,NORMAL,9000,2018-05-02,"[{'status': 'MODEL_NORMAL', 'original_price': ...",3,0.10,[],23
0,"[{'attribute_name': '品牌', 'is_mandatory': True...",11460,0,2017-11-30,TWD,5,歐美品質 優質線材 適用於各種3孔梅花孔 電源 適配器 充電器 AC線 美規 使用優質銅線製...,False,[http://f.shopee.tw/file/9d77a1606440bcfe3ef12...,725349781,...,0,23070969,NORMAL,1000,2018-03-10,[],25,0.10,[],176
0,"[{'attribute_name': '品牌', 'is_mandatory': True...",11460,0,2017-11-30,TWD,5,歐美品質 優質線材 適用於各種3孔梅花孔 電源 適配器 充電器 AC線 英規 使用優質銅線製...,False,[http://f.shopee.tw/file/fea0a8aa3e669b532f5eb...,725351767,...,0,23070969,NORMAL,1000,2018-02-28,[],6,0.10,[],176
0,"[{'attribute_name': '品牌', 'is_mandatory': True...",11460,0,2017-11-30,TWD,5,歐美品質 優質線材 適用於各種3孔梅花孔 電源 適配器 充電器 AC線 歐規 使用優質銅線製...,False,[http://f.shopee.tw/file/42eb3e3452d5f3830720a...,725352772,...,0,23070969,NORMAL,1000,2018-02-28,[],11,0.10,[],176
0,[],5794,2,2017-06-01,TWD,5,廠家直銷高品質超級本適配器19V 1.75A(4.0*1.35) #電源適配器 #適配器 #...,False,[http://f.shopee.tw/file/ca07d670441c20e94b54c...,288208978,...,2,23070969,NORMAL,999,2018-02-28,[],92,0.35,[],358
0,[],5794,1,2017-06-02,TWD,5,Surface 3 平板電腦電源適配器 充電器5.2V2.5A #電源適配器 #適配器 #充...,False,[http://f.shopee.tw/file/7c916db4f2d8876bfe8a7...,289817750,...,1,23070969,NORMAL,1000,2018-02-28,[],65,0.35,[],357
0,[],5794,0,2017-06-02,TWD,5,19V 3.42A筆記本電源適配器 旅行可擕式充電器65W(3.0*1.0小口) #電源適配...,False,[http://f.shopee.tw/file/a71d4c0b2c1e5cd389e04...,289817643,...,2,23070969,NORMAL,1000,2018-02-28,[],44,0.35,[],357
0,[],5794,0,2017-06-02,TWD,5,yoga 900筆記本電源適配器超級本電腦充電器20V2A(USB介面) #電源適配器 #適...,False,[http://f.shopee.tw/file/690654d1793a9ba7c49e2...,289817813,...,1,23070969,NORMAL,1000,2018-02-28,[],35,0.35,[],357


In [553]:
# 选上线超过60天仍然0销量的产品
item_detail_list = item_detail_list[item_detail_list['days_since_live'] >= 60]
item_detail_list = item_detail_list[item_detail_list['sales'] == 0]
item_detail_list = item_detail_list.sort_values(by='days_since_live', ascending=False)
item_detail_list

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,sales,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636440,...,0,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,692636426,...,0,23070969,NORMAL,1000,2018-02-28,[],5,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,0,23070969,NORMAL,1001,2018-02-28,[],20,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,0,23070969,NORMAL,1000,2018-02-28,[],4,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,0,23070969,NORMAL,1001,2018-02-28,[],2,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,0,23070969,NORMAL,1001,2018-02-28,[],18,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,0,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,0,23070969,NORMAL,1001,2018-02-28,[],142,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,0,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],189
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,0,23070969,NORMAL,1001,2018-02-28,[],19,0.35,[],189


In [554]:
pd.DataFrame(item_detail_list['attributes'])

,attributes
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals..."


In [555]:
def retrieve_dict_and_convert_to_df_column(dataframe, column_name, selected_key_list, converted_column_name):

    edited_column_list = []

    for index, record in dataframe.iterrows():
        to_edit_column = pd.DataFrame(record[column_name])
        try:
            to_edit_column = to_edit_column[selected_key_list]
            to_edit_column_dict = to_edit_column.to_dict('records')
        except:
            to_edit_column_dict = '[]'
        edited_column_list.append(to_edit_column_dict)
    
    dataframe[converted_column_name] = edited_column_list
    return dataframe

In [556]:
def retrieve_image_and_convert_to_dict(dataframe, column_name, converted_column_name):
    edited_column_list = []
    for index, record in dataframe.iterrows():
        img_url_dict_list = []
        to_edit_column = pd.DataFrame(record[column_name])
        for index, img_url in to_edit_column.iterrows():
            img_url_dict = {'url': img_url[0]}
            img_url_dict_list.append(img_url_dict)
        edited_column_list.append(img_url_dict_list)
    dataframe[converted_column_name] = edited_column_list
    return dataframe

In [557]:
column_name = 'attributes'
selected_key_list = ['attribute_id', 'attribute_value']
converted_column_name = 'edited attributes'

retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636440,...,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,692636426,...,23070969,NORMAL,1000,2018-02-28,[],5,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,23070969,NORMAL,1001,2018-02-28,[],20,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,23070969,NORMAL,1000,2018-02-28,[],4,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,23070969,NORMAL,1001,2018-02-28,[],2,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,23070969,NORMAL,1001,2018-02-28,[],18,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,23070969,NORMAL,1001,2018-02-28,[],142,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,23070969,NORMAL,1001,2018-02-28,[],19,0.35,[],189,"[{'attribute_id': 4590, 'attribute_value': '筆記..."


In [558]:
item_attribute_list = retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)
# item_attribute_list.to_csv('D://item_attribute_list.csv', sep=',')

In [559]:
# 把attribute_id改为attributes_id
for value in item_detail_list['edited attributes']:
    for dict_value in value:
        dict_value['attributes_id'] = dict_value.pop('attribute_id')
        dict_value['value'] = dict_value.pop('attribute_value')
    # print(value)
item_detail_list

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636440,...,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,692636426,...,23070969,NORMAL,1000,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,23070969,NORMAL,1001,2018-02-28,[],20,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,23070969,NORMAL,1000,2018-02-28,[],4,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,23070969,NORMAL,1001,2018-02-28,[],2,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,23070969,NORMAL,1001,2018-02-28,[],18,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,23070969,NORMAL,1001,2018-02-28,[],142,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,23070969,NORMAL,1001,2018-02-28,[],19,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'}]"


In [560]:
# 根据每个分类，找到必填的attributes，然后填上
def get_category_attributes(category_id):
    
    # Current time stamp
    timestamp = int(time.time())
    
    payload = {"partner_id":partner_id,
               "shopid": shopid,
               "timestamp": timestamp,
               "category_id": category_id}

    url = 'https://partner.shopeemobile.com/api/v1/item/attributes/get'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)
    # 打印原始Json
    parsed = json.loads(r.text)
    # 漂亮打印Json
    attribute_list = json_normalize(parsed['attributes'])
    attribute_list['category_id'] = category_id
    return attribute_list

# 找店铺分类
def get_category_info():
    
    # Current time stamp
    timestamp = int(time.time())
    
    payload = {"partner_id":partner_id,
               "shopid": shopid,
               "timestamp": timestamp}

    url = 'https://partner.shopeemobile.com/api/v1/item/categories/get'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)
    # 打印原始Json
    parsed = json.loads(r.text)
    # 漂亮打印Json
    category_list = json_normalize(parsed['categories'])
    return category_list

attribute_list = get_category_attributes(5794)
attribute_list = attribute_list[attribute_list['is_mandatory'] == True]
attribute_list

,attribute_id,attribute_name,attribute_type,input_type,is_mandatory,options,category_id
1,9547,品牌,STRING_TYPE,COMBO_BOX,True,"[自有品牌, 360 奇虎, 3D SYSTEMS, 3Dconnexion, 3Glass...",5794


In [561]:
# 看店铺所有分类的必填attributes
category_id_list = item_detail_list['category_id'].drop_duplicates()
category_id_list

0     5794
0    11460
Name: category_id, dtype: int64

In [562]:
attribute_full_list = pd.DataFrame()
for category_id in category_id_list:
    print(int(category_id))
    category_attribute = get_category_attributes(int(category_id))
    attribute_full_list = attribute_full_list.append(category_attribute)
    
attribute_full_list

5794
11460


,attribute_id,attribute_name,attribute_type,input_type,is_mandatory,options,category_id
0,4590,類型,STRING_TYPE,COMBO_BOX,False,"[USB Hub, 風扇, 濾藍光眼鏡, 搖桿, 延長線, 繪圖板, 充電電池]",5794
1,9547,品牌,STRING_TYPE,COMBO_BOX,True,"[自有品牌, 360 奇虎, 3D SYSTEMS, 3Dconnexion, 3Glass...",5794
0,9547,品牌,STRING_TYPE,COMBO_BOX,True,"[自有品牌, 360 奇虎, 3D SYSTEMS, 3Dconnexion, 3Glass...",11460


In [563]:
# 给每个必填attribute提供default value
attribute_default_value_list = [
    [5794, 9547, '握力'],
    [11460, 9547, '握力']
]

attribute_default_value_columns = ['category_id', 'attribute_id', 'value']

attribute_default_value_df = pd.DataFrame(attribute_default_value_list, columns=attribute_default_value_columns)
attribute_default_value_df

,category_id,attribute_id,value
0,5794,9547,握力
1,11460,9547,握力


In [564]:
attribute_full_list = pd.merge(attribute_full_list, attribute_default_value_df, how='left',
                               left_on=['category_id', 'attribute_id'], right_on=['category_id', 'attribute_id'])
attribute_full_list                                                                                  

,attribute_id,attribute_name,attribute_type,input_type,is_mandatory,options,category_id,value
0,4590,類型,STRING_TYPE,COMBO_BOX,False,"[USB Hub, 風扇, 濾藍光眼鏡, 搖桿, 延長線, 繪圖板, 充電電池]",5794,NaN
1,9547,品牌,STRING_TYPE,COMBO_BOX,True,"[自有品牌, 360 奇虎, 3D SYSTEMS, 3Dconnexion, 3Glass...",5794,握力
2,9547,品牌,STRING_TYPE,COMBO_BOX,True,"[自有品牌, 360 奇虎, 3D SYSTEMS, 3Dconnexion, 3Glass...",11460,握力


In [565]:
# 检查每个产品的attributes
# 1.检查现有的attributes是否在表里；如果不存在，要删除
# 2.如果缺了必填的attributes，要补充回来
for index, value in item_detail_list.iterrows():
    # 拿到每个产品的cat id
    category_id = value['category_id']
    # 创建一个空list，装这个产品的所有现有attributes
    existing_attribute = []
    
    # for loop所有这个产品的attributes，放在existing_attribute
    for dict_value in value['edited attributes']:
        existing_attribute.append(dict_value['attributes_id'])
    # print(existing_attribute)
    
    # 把这个cat的必填attribute拉出来
    cat_mandatory_attribute_df = (attribute_full_list[(attribute_full_list['category_id'] == category_id)
                                                       & (attribute_full_list['is_mandatory'] == True)])
    # print((cat_mandatory_attribute_list))
    
    # 把这个cat的非必填attribute拉出来
    cat_optional_attribute_df = (attribute_full_list[(attribute_full_list['category_id'] == category_id)
                                                       & (attribute_full_list['is_mandatory'] == False)])
    # print((cat_optional_attribute_list))
    
    # 判断必填attributes是不是在现有attributes内
    for index, attr in cat_mandatory_attribute_df.iterrows():
        mandatory_attribute_id = attr['attribute_id']
        mandatory_attribute_value = attr['value']
        
        if mandatory_attribute_id not in existing_attribute:
            # print('attr: ' + str(mandatory_attribute_id) + ' is not in mandatory_attribute_list.')
            attr_dict = {'attributes_id': mandatory_attribute_id,
                         'value': mandatory_attribute_value}
            value['edited attributes'].append(attr_dict)
    
    # 判断现有非必填attributes内是否有不符合的非必填attributes
    for attr in existing_attribute:
        # print(attr)
        if attr not in list(cat_mandatory_attribute_df['attribute_id']):
            if attr not in list(cat_optional_attribute_df['attribute_id']):
                print('attr: ' + str(attr) + ' is not in optional_attribute_list.')
                # 删除这个不符合的非必填项
                for dict_value in value['edited attributes']:
                    if dict_value['attributes_id'] == attr:
                        del dict_value
            
item_detail_list.to_csv("D://item_detail_list.csv", sep=',')

In [566]:
column_name = 'variations'
selected_key_list = ['name', 'stock', 'price', 'variation_sku']
converted_column_name = 'edited variations'

retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,status,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes,edited variations
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636440,...,NORMAL,1001,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,692636426,...,NORMAL,1000,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,NORMAL,1001,2018-02-28,[],20,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,NORMAL,1000,2018-02-28,[],4,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,NORMAL,1001,2018-02-28,[],2,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,NORMAL,1001,2018-02-28,[],18,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,NORMAL,1001,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,NORMAL,1001,2018-02-28,[],142,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,NORMAL,1001,2018-02-28,[],10,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,NORMAL,1001,2018-02-28,[],19,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[]


In [567]:
column_name = 'logistics'
selected_key_list = ['logistic_id', 'enabled']
converted_column_name = 'edited logistics'

retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes,edited variations,edited logistics
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636440,...,1001,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,692636426,...,1000,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,1001,2018-02-28,[],20,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,1000,2018-02-28,[],4,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,1001,2018-02-28,[],2,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,1001,2018-02-28,[],18,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,1001,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,1001,2018-02-28,[],142,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,1001,2018-02-28,[],10,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,1001,2018-02-28,[],19,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."


In [568]:
column_name = 'images'
converted_column_name = 'edited images'

retrieve_image_and_convert_to_dict(item_detail_list, column_name, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,update_time,variations,views,weight,wholesales,days_since_live,edited attributes,edited variations,edited logistics,edited images
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636440,...,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/835be51e6df5...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,692636426,...,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/46a57a5505e2...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,2018-02-28,[],20,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/5a41b72bd294...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,2018-02-28,[],4,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/218e48e2be57...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,2018-02-28,[],2,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/310b4d24f968...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,2018-02-28,[],18,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/4e932239e29d...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,2018-02-28,[],5,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/2997d787d057...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,2018-02-28,[],142,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/8e69810052f7...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,2018-02-28,[],10,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",[{'url': 'http://f.shopee.tw/file/56c04008c69b...
0,"[{'attribute_name': '類型', 'is_mandatory': Fals...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,2018-02-28,[],19,0.35,[],189,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...",[],"[{'logistic_id': 38011

In [569]:
selected_columns = ["category_id","name","description","price","stock","item_sku","edited variations","images","edited attributes","edited logistics","weight","days_to_ship", 'item_id', 'edited images']
edited_item_detail_list = item_detail_list[selected_columns]
edited_item_detail_list

,category_id,name,description,price,stock,item_sku,edited variations,images,edited attributes,edited logistics,weight,days_to_ship,item_id,edited images
0,5794,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,199.0,1001,32298200001,[],[http://f.shopee.tw/file/835be51e6df50f35a7f67...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636440,[{'url': 'http://f.shopee.tw/file/835be51e6df5...
0,5794,歐美品質 適用於華碩 X201E X200T S200 S200E S200L X202E ...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1000,32808660934,[],[http://f.shopee.tw/file/46a57a5505e23df0f05b2...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636426,[{'url': 'http://f.shopee.tw/file/46a57a5505e2...
0,5794,歐美品質 適用於華碩 A8 F8 A43S F80 F82 K40 A45 X81 M50 ...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32297003488,[],[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636396,[{'url': 'http://f.shopee.tw/file/5a41b72bd294...
0,5794,歐美品質 適用於華碩 S200 S200E S220 X200T X201E X202E F...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1000,32435638867,[],[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636455,[{'url': 'http://f.shopee.tw/file/218e48e2be57...
0,5794,歐美品質 適用於華碩 M9V R1 S1 S2 S3 S5 A3 A600 F3 X55 A...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,329.0,1001,32743178926,[],[http://f.shopee.tw/file/310b4d24f9688ff068381...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636450,[{'url': 'http://f.shopee.tw/file/310b4d24f968...
0,5794,歐美品質 適用於華碩 UX21 UX21E UX31 UX31E UX21A UX31A U...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32441993728,[],[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636463,[{'url': 'http://f.shopee.tw/file/4e932239e29d...
0,5794,歐美品質 適用於華碩 S200 S200E S220 X200T X201E X202E F...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32441829684,[],[http://f.shopee.tw/file/2997d787d0579741d5399...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636465,[{'url': 'http://f.shopee.tw/file/2997d787d057...
0,5794,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,199.0,1001,32296958243,[],[http://f.shopee.tw/file/8e69810052f7edc68eedb...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692640311,[{'url': 'http://f.shopee.tw/file/8e69810052f7...
0,5794,歐美品質 適用華碩 S200E S200 X200T X201E X202E F201E Q...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32486535722,[],[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692639116,[{'url': 'http://f.shopee.tw/file/56c04008c69b...
0,5794,歐美品質 適用於華碩 UX21 UX31 UX31E UX31K UX32 UX42 筆記型...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,199.0,1001,32297550062,[],[http://f.shopee.tw/file/835be51e6df50f35a7f67...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636460,[{'url': 'http://f.shopee.tw/file/835be51e6df5...


In [570]:
# 测试
test_item_detail_list = edited_item_detail_list[:1]
test_item_detail_list

,category_id,name,description,price,stock,item_sku,edited variations,images,edited attributes,edited logistics,weight,days_to_ship,item_id,edited images
0,5794,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E ...,199.0,1001,32298200001,[],[http://f.shopee.tw/file/835be51e6df50f35a7f67...,"[{'attributes_id': 4590, 'value': '筆記型電腦適配器'},...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636440,[{'url': 'http://f.shopee.tw/file/835be51e6df5...


In [571]:
def delete_product(delete_product_id):
    # Current time stamp
    timestamp = int(time.time())
    
    # payload
    payload = {
               "item_id": delete_product_id,
               "partner_id": partner_id,
               "shopid": shopid,
               "timestamp": timestamp
               }

    print(payload)
    
    url = 'https://partner.shopeemobile.com/api/v1/item/delete'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)
    # 打印原始Json
    parsed = json.loads(r.text)
    logging.info('Product is deleted.')

In [572]:
def add_product(category_id,
               name,
               description,
               price,
               stock,
               item_sku,
               edited_variations,
               edited_images,
               edited_attributes,
               edited_logistics,
               weight
               ):
    # Current time stamp
    timestamp = int(time.time())
    
    # payload
    payload = {
                "category_id": category_id,
                "name": name,
                "description": description,
                "price": price,
                "stock": stock,
                "item_sku": item_sku,
                "images": edited_images,
                "weight": weight,
                "logistics": edited_logistics,
                # "days_to_ship": days_to_ship,
                "partner_id": partner_id,
                "shopid": shopid,
                "timestamp": timestamp
               }
    
    # 判断条件是否为空
    if edited_variations != '[]':
        payload['variations'] = edited_variations
        
    if edited_attributes != '[]':
        payload['attributes'] = edited_attributes
        
    print(payload)

    url = 'https://partner.shopeemobile.com/api/v1/item/add'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)  
    # 打印原始Json
    parsed = json.loads(r.text)
    logging.info('Product is added.')
    
    # 判断是否有图片上传失败；如果失败要重传
    try:
        item_id = parsed['item_id']
        fail_image_list = parsed['fail_image']
        logging.info('Try to upload failed images.')
        add_product_iamge(item_id, fail_image_list)
    except:
        pass

In [573]:
def add_product_image(item_id, images):
    # Current time stamp
    timestamp = int(time.time())
    
    # payload
    payload = {
                "item_id": item_id,
                "images": images,
                "partner_id": partner_id,
                "shopid": shopid,
                "timestamp": timestamp
               }

    url = 'https://partner.shopeemobile.com/api/v1/item/img/add'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)  
    # 打印原始Json
    parsed = json.loads(r.text)
    logging.info('Images are added.')

In [574]:
for index, item in test_item_detail_list.iterrows():
    category_id = item[0]
    name = item[1]
    description = item[2]
    price = item[3]
    stock = item[4]
    item_sku = item[5]
    edited_variations = item[6]
    images = item[7]
    edited_attributes = item[8]
    edited_logistics = item[9]
    weight = item[10]
    # days_to_ship = 3
    product_id = item[12]
    edited_images = item[13]
    
    # print(images)
    
    # delete product
    # delete_product(product_id)
    
    # add product
    add_product(category_id,
               name,
               description,
               price,
               stock,
               item_sku,
               edited_variations,
               edited_images,
               edited_attributes,
               edited_logistics,
               weight
               )

{'category_id': 5794, 'name': '歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E X202E 電源適配器', 'description': '歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E X202E 電源適配器 4.0mm * 1.35mm 19V 1.75A 33W\n\n我司設廠于大陸廣東省廣州市，專職批量生產筆記本電源適配器，主要出口歐美市場（英美法德荷西加拿大等），品質有保證，已通過各項電源認證。\n\n因生產場地位於大陸廣州，需經過蝦皮深圳轉運倉運往台灣，需時約7-10天，暫無加急服務，請有需要拍下的親們耐心等待！\n\n#適配器 #充電器\n#華碩適配器 #華碩充電器\n#筆記型電腦適配器 #筆記型電腦充電器\n#華碩\n#華碩筆記型電腦\n#華碩電腦\n#電源適配器 #適配器 #充電器 #筆記本充電器 #筆記本適配器 #筆電充電器 #筆電適配器', 'price': 199.0, 'stock': 1001, 'item_sku': '32298200001', 'images': [{'url': 'http://f.shopee.tw/file/835be51e6df50f35a7f670b13d93e3fc'}, {'url': 'http://f.shopee.tw/file/88c6c7922239e9dcf161e7d11ddc0c5b'}, {'url': 'http://f.shopee.tw/file/eed05c54191a76fc199a427fc9b38935'}], 'weight': 0.35, 'logistics': [{'logistic_id': 38011, 'enabled': True}, {'logistic_id': 38012, 'enabled': True}], 'partner_id': 12155, 'shopid': 23070969, 'timestamp': 1527235678, 'attributes': [{'attributes_id': 4590, 'value': '筆記型電腦適配器'}, {'attributes_id': 9547, 'value': '握力'

 2018-05-25 16:08:00,307 - INFO - {'item_id': 1190276615, 'msg': 'Add item success', 'item': {'logistics': [{'logistic_name': '圓通-黑貓宅配', 'is_free': False, 'estimated_shipping_fee': 55.0, 'logistic_id': 38011, 'enabled': True}, {'logistic_name': '圓通-711超取', 'is_free': False, 'estimated_shipping_fee': 55.0, 'logistic_id': 38012, 'enabled': True}], 'original_price': 199.0, 'cmt_count': 0, 'weight': 0.35, 'shopid': 23070969, 'currency': 'TWD', 'create_time': 1527235694, 'likes': 0, 'images': ['http://f.shopee.tw/file/9a5e782d91c342e8f340bcc40fee3906', 'http://f.shopee.tw/file/d666d01b693d05e0cc6eb487112ff820', 'http://f.shopee.tw/file/99b30f15306f601be08b829290718f48'], 'days_to_ship': 3, 'stock': 1001, 'status': 'NORMAL', 'update_time': 1527235694, 'description': '歐美品質 適用於華碩 S200 S200E X200T F201E Q200E X201E X202E 電源適配器 4.0mm * 1.35mm 19V 1.75A 33W\n\n我司設廠于大陸廣東省廣州市，專職批量生產筆記本電源適配器，主要出口歐美市場（英美法德荷西加拿大等），品質有保證，已通過各項電源認證。\n\n因生產場地位於大陸廣州，需經過蝦皮深圳轉運倉運往台灣，需時約7-10天，暫無加急服務，請有需要拍下的親們耐心等待！\n\n#適配器 #充

In [575]:
# item_detail_list.to_csv("D://item_detail_list.csv", sep=',')
# edited_item_detail_list.to_csv("D://edited_item_detail_list.csv", sep=',')